In [1]:
import json
from pprint import pprint
from transformers import AutoModelForCausalLM, AutoTokenizer
from tqdm import tqdm
import torch
import random

In [2]:
tokenizer = AutoTokenizer.from_pretrained('mistralai/Mistral-7B-Instruct-v0.2')
# model = AutoModelForCausalLM.from_pretrained(
#     'mistralai/Mistral-7B-Instruct-v0.2',
#     use_flash_attention_2 = True,
#     torch_dtype = torch.float16
# )
# tokenizer.pad_token = tokenizer.eos_token
# model.cuda()

### Image

In [3]:
image = []

with open('dataset/llava/blip_laion_cc_sbu_558k.translated.jsonl') as fopen:
    for x in fopen:
        image.append(json.loads(x))
        
len(image)

558128

In [4]:
image[0]

{'id': '004539375',
 'image': '00453/004539375.jpg',
 'conversations': [{'from': 'human',
   'value': 'Render a clear and concise summary of the photo.\n<image>',
   'value_ms': 'Render ringkasan foto yang jelas dan ringkas.\n<imej>'},
  {'from': 'gpt',
   'value': 'select luxury furniture 3 - inch gel memory foam mattress topper',
   'value_ms': 'pilih perabot mewah 3 inci memori gel buih tilam topper'}]}

In [8]:
batch_size = 100
prompt_list = []
selected_image = set()

with open('llava-multi-images-template.jsonl', 'w') as fopen:
    for x in tqdm(range(0, 100000, batch_size)):
        
        prompts = []
        images0 = []
        images1 = []
        for _ in range(batch_size):
            image_ids = random.sample(list(set(range(len(image))) - selected_image), 2)
            selected_image |= set(image_ids)

            messages = [{'role': 'user',
                              'content': f"""
            Picture 1: {image[image_ids[0]]['conversations'][1]['value']} 
            Picture 2: {image[image_ids[1]]['conversations'][1]['value']}
            What is related between picture 1 and picture 2."""}]

            prompt = tokenizer.apply_chat_template(messages, tokenize = False)
            prompts.append(prompt)
            images0.append((image[image_ids[0]]['conversations'][1]['value'], image[image_ids[0]]['image']))
            images1.append((image[image_ids[1]]['conversations'][1]['value'], image[image_ids[1]]['image']))
            
        d = {
            'prompts': prompts,
            'images0': images0,
            'images1': images1,
        }
        fopen.write(f'{json.dumps(d)}\n')

100%|██████████| 1000/1000 [1:26:33<00:00,  5.19s/it]
